# Ron Inference

In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pytz

frequency = "1d"
start_time = "2012-01-01"
end_time = (datetime.now(pytz.timezone('US/Pacific'))).strftime('%Y-%m-%d')

tickers = ["BTC-CAD"]

# Retrieve and concatenate historical data in one step
df = pd.concat(
    [yf.download(ticker, start=start_time, end=end_time, interval=frequency)[['High', 'Low']].add_prefix(f"{ticker}_") for ticker in tickers],
    axis=1
)

# Fill missing data
df.fillna(method='ffill', inplace=True)

[*********************100%%**********************]  1 of 1 completed
C:\Users\falty\AppData\Local\Temp\ipykernel_1520\1692606717.py:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [2]:
# Assuming df is your DataFrame after resampling and you've already dropped NA values
df.dropna(inplace=True)

# Add an assertion to ensure there are no NA values in the DataFrame
assert df.isnull().sum().sum() == 0, "DataFrame contains NA values"

df

,BTC-CAD_High,BTC-CAD_Low
Date,,
2014-09-17,513.446411,497.302246
2014-09-18,502.728729,452.183655
2014-09-19,468.607666,419.985870
2014-09-20,463.974731,427.350739
2014-09-21,452.060120,430.965698
...,...,...
2024-03-07,91916.906250,85369.171875
2024-03-08,91648.546875,88748.687500
2024-03-09,94348.257812,89181.023438


In [3]:
import pandas as pd
import numpy as np
import torch

from gluonts.dataset.common import ListDataset
from gluonts.torch.model.tft import TemporalFusionTransformerEstimator
from gluonts.transform.feature import MissingValueImputation

torch.set_float32_matmul_precision('high')
                                   
# Assuming df is your DataFrame with the data
target_column_high = 'BTC-CAD_High' 
target_column_low = 'BTC-CAD_Low' 

# Ensure the DataFrame's index is a datetime index and set the frequency explicitly if needed
df.index = pd.to_datetime(df.index)
freq = "D"  # Set the frequency of your data, e.g., 'D' for daily. Adjust as needed.
df = df.asfreq(freq)

# Define the prediction length
prediction_length = 1  # Set your prediction length
context_length = 60

In [4]:
import os
import torch

model_high = 'ron_high_huge.pth'
model_low = 'ron_low_huge.pth'

predictor_high = torch.load(model_high)
predictor_low = torch.load(model_low)

In [5]:
import pandas as pd
import numpy as np
from gluonts.dataset.common import ListDataset

# Rolling window prediction function
def perform_rolling_prediction(df, predictor, target_column, context_length, prediction_length):
    rolling_predictions = []
    prediction_dates = []
    temp_df = df.copy()  # Create a copy of the dataframe to modify

    # Create dates for predictions
    start_prediction_date = temp_df.index.max() + pd.Timedelta(1, unit='D')
    prediction_dates = pd.date_range(start=start_prediction_date, periods=30, freq='D')
    
    for i, prediction_date in enumerate(prediction_dates):
        end_idx = len(temp_df) - prediction_length + i
        test_data = ListDataset([
            {
                "start": temp_df.index[0],
                "target": temp_df[target_column][:end_idx].values
            }
        ], freq='D')

        forecast = next(predictor.predict(test_data))
        predicted_value = forecast.quantile(0.5)[-1]
        rolling_predictions.append(predicted_value)
        
        # Append the predicted value for future rolling windows
        new_row = pd.DataFrame({target_column: [predicted_value]}, index=[prediction_date])
        temp_df = pd.concat([temp_df, new_row])
    
    return prediction_dates, rolling_predictions

# Perform rolling predictions for the next 30 days
prediction_dates_high, rolling_predictions_high = perform_rolling_prediction(df, predictor_high, target_column_high, context_length, prediction_length)
prediction_dates_low, rolling_predictions_low = perform_rolling_prediction(df, predictor_low, target_column_low, context_length, prediction_length)


# Print the rolling predictions
print("High Rolling Predictions for the next 30 days:")
for date, prediction in zip(prediction_dates_high, rolling_predictions_high):
    print(f"{date.strftime('%Y-%m-%d')}: {prediction}")

# Print the rolling predictions
print("Low Rolling Predictions for the next 30 days:")
for date, prediction in zip(prediction_dates_low, rolling_predictions_low):
    print(f"{date.strftime('%Y-%m-%d')}: {prediction}")

High Rolling Predictions for the next 30 days:
2024-03-12: 91310.640625
2024-03-13: 92620.109375
2024-03-14: 95395.921875
2024-03-15: 95084.265625
2024-03-16: 93672.0234375
2024-03-17: 91378.0625
2024-03-18: 91342.78125
2024-03-19: 92202.859375
2024-03-20: 90631.3125
2024-03-21: 87780.6640625
2024-03-22: 86892.328125
2024-03-23: 86496.40625
2024-03-24: 88076.96875
2024-03-25: 88421.546875
2024-03-26: 86084.40625
2024-03-27: 85754.8203125
2024-03-28: 85784.65625
2024-03-29: 91195.0390625
2024-03-30: 88689.1953125
2024-03-31: 88975.21875
2024-04-01: 91143.5234375
2024-04-02: 90667.7421875
2024-04-03: 90634.453125
2024-04-04: 90251.796875
2024-04-05: 90590.5859375
2024-04-06: 91484.46875
2024-04-07: 92733.1796875
2024-04-08: 92400.2890625
2024-04-09: 92813.34375
2024-04-10: 93058.6484375
Low Rolling Predictions for the next 30 days:
2024-03-12: 88542.609375
2024-03-13: 88813.84375
2024-03-14: 90737.140625
2024-03-15: 89708.203125
2024-03-16: 89696.984375
2024-03-17: 90243.2109375
2024-03-

In [6]:
import pandas as pd

def update_predictions(prediction_dates, high_predictions, low_predictions):
    # Load the existing predictions, or initialize a new DataFrame if not present
    try:
        predictions = pd.read_parquet('average_predictions.parquet')
    except FileNotFoundError:
        print("File not found. Creating a new file.")
        # Initialize an empty DataFrame with a fixed range of columns (Days_Before)
        max_days_before = 30  # Adjust this value based on the maximum days range you expect
        days_before_columns = list(range(1, max_days_before + 1))
        predictions = pd.DataFrame(columns=days_before_columns)

    # Calculate the earliest date from the existing data
    reference_date = pd.to_datetime(prediction_dates[0]) - pd.Timedelta(days=1)

    # Combine the high and low predictions and calculate their average
    new_data = pd.DataFrame({
        'Date': prediction_dates,
        'High_Prediction': high_predictions,
        'Low_Prediction': low_predictions
    })
    new_data['Average_Prediction'] = new_data[['High_Prediction', 'Low_Prediction']].mean(axis=1)

    # Set 'Date' as the index and convert it to datetime
    new_data.set_index('Date', inplace=True)
    new_data.index = pd.to_datetime(new_data.index)

    # Calculate 'Days_Before' for the new data
    new_data['Days_Before'] = (new_data.index - reference_date).days

    # Update the existing DataFrame with new predictions
    for date, row in new_data.iterrows():
        day_before = row['Days_Before']
        if day_before in predictions.columns:
            predictions.at[date, day_before] = row['Average_Prediction']

    # Save the updated dataframe to a parquet file
    predictions.to_parquet('average_predictions.parquet')

    return predictions

# Example usage:
# Assuming you have the results from your rolling prediction functions
updated_predictions = update_predictions(prediction_dates_high, rolling_predictions_high, rolling_predictions_low)

# Print the updated predictions
updated_predictions


Days_Before,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
Date,,,,,,,,,,,,,,,,,,,,,
2024-03-08,87807.117188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-03-09,89343.546875,88224.140625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-03-10,87882.968750,89957.140625,88574.820312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-03-11,NaN,85524.914062,88300.453125,87315.273438,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-03-12,89926.625000,NaN,86741.359375,87643.093750,86493.265625,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-03-13,NaN,90716.976562,NaN,88036.171875,89522.421875,88061.250000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-03-14,NaN,NaN,93066.531250,NaN,90395.375000,90762.218750,89333.359375,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-03-15,NaN,NaN,NaN,92396.234375,NaN,89817.171875,92194.031250,90585.828125,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-03-16,NaN,NaN,NaN,NaN,91684.500000,NaN,91080.109375,92572.914062,91478.484375,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
